In [6]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter

In [7]:
import matplotlib.pyplot as plt

import stuff

In [8]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt')

In [9]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282


read data in + merge data

In [10]:
data.columns = data.columns.str.replace(' ','')

remove spaces from exits column

In [11]:
data["DATETIME"] = data["DATE"] + ' ' + data["TIME"]

In [12]:
data['DATETIME'] = pd.to_datetime(data.DATETIME, format='%m/%d/%Y %H:%M:%S')

create datetime and set format

In [13]:
data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [14]:
data['WEEKDAY']=data.DATETIME.apply(lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend')

#same as above in 2 steps
#weekdaycheck = lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend'
#data['WEEKDAY']=data.DATETIME.apply(weekdaycheck)


In [15]:
daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

whatdayofweek = lambda x: daysofweek[x.weekday()]


data['DAYOFWEEK'] = data.DATETIME.apply(whatdayofweek)
#data['DAYOFWEEKVALUE'] = data.DATETIME.apply(dayofweekvalue)

create day of week by applying function to DATETIME

In [16]:
data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS
data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###for considering deltas as time(i+1)-time(i)

create deltas by shifting from one above

In [17]:
#data['dENTRIES'] = data.ENTRIES - data.ENTRIES.shift(1)
#data['dEXITS'] = data.EXITS - data.EXITS.shift(1)
#data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###this is for considering deltas as time(i)-time(i-1)

In [18]:
datatest = data[(data.HOUR ==0)|(data.HOUR ==1)]
# same as mask

filter based on time we care about

In [19]:
datatest2 = datatest[(datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)]
# mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)


filter a second time based on change in deltas to account for switching machine/station etc.  
Lose out on 29835 - 29522 = 313 total entries  
313 is 1.05% of our data that we lose

In [20]:
final_weekday_or_end = datatest2.groupby([datatest2.WEEKDAY, datatest2.STATION]).sum()

In [21]:
weekend_or_day_final =  (final_weekday_or_end
    .reset_index()
    .sort_values(["WEEKDAY", "dTOTAL"], ascending=[True, False])
)

In [22]:
weekend_or_day_final = weekend_or_day_final.drop(['ENTRIES', 'EXITS', 'HOUR', 'dENTRIES', 'dEXITS'],1)

In [23]:
for r in range(weekend_or_day_final.shape[0]):
    if weekend_or_day_final.ix[r,'WEEKDAY'] =='Weekday':
        weekend_or_day_final.ix[r,'dTOTALACTUAL'] = weekend_or_day_final.ix[r, 'dTOTAL']/5
    else:
        weekend_or_day_final.ix[r,'dTOTALACTUAL'] = weekend_or_day_final.ix[r, 'dTOTAL']/2

In [24]:
weekend_or_day_final = weekend_or_day_final.drop(['dTOTAL'],1)


In [25]:
weekend_or_day_final.head()

,WEEKDAY,STATION,dTOTALACTUAL
345,Weekday,TIMES SQ-42 ST,3975.0
59,Weekday,34 ST-HERALD SQ,2582.4
251,Weekday,JKSN HT-ROOSVLT,2492.8
61,Weekday,34 ST-PENN STA,2447.4
9,Weekday,125 ST,2416.4


weekend_or_day_final is the FINAL dataframe for weekday vs weekend

In [26]:
finaldayofweek = datatest2.groupby([datatest2.DAYOFWEEK,datatest2.STATION]).sum()

In [27]:
day_of_week = finaldayofweek.reset_index()

In [28]:
whatdayindex = lambda x: daysofweek.index(x)

In [29]:
day_of_week['DAYOFWEEKVALUE'] = day_of_week.DAYOFWEEK.apply(whatdayindex)  


day_of_week_final = day_of_week.sort_values(['DAYOFWEEKVALUE','dTOTAL'], ascending = [True, False])
#data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [30]:
day_of_week_final = day_of_week_final.drop(['ENTRIES', 'EXITS', 'HOUR', 'dENTRIES', 'dEXITS', 'DAYOFWEEKVALUE'],1)

#df = df.drop('column_name', 1)

In [31]:
day_of_week_final[day_of_week_final['DAYOFWEEK'] == 'Monday'].head(10)

,DAYOFWEEK,STATION,dTOTAL
713,Monday,TIMES SQ-42 ST,3943.0
377,Monday,125 ST,2314.0
429,Monday,34 ST-PENN STA,2184.0
619,Monday,JKSN HT-ROOSVLT,2111.0
383,Monday,145 ST,1841.0
427,Monday,34 ST-HERALD SQ,1776.0
579,Monday,FLUSHING-MAIN,1552.0
595,Monday,GRD CNTRL-42 ST,1453.0
475,Monday,86 ST,1389.0
545,Monday,CHURCH AV,1355.0


day_of_week_final is FINAL dataframe for each day of week

In [104]:
import gmaps
import numpy as np


In [105]:
# setting up dataframe of coordinates

In [106]:
day_of_week_final['dScale'] = (day_of_week_final['dTOTAL'] / 250) ** 2

In [145]:
day_of_week_final[day_of_week_final['DAYOFWEEK'] == 'Monday'].head(10)

,DAYOFWEEK,STATION,dTOTAL,dScale
713,Monday,TIMES SQ-42 ST,3943.0,248.755984
377,Monday,125 ST,2314.0,85.673536
429,Monday,34 ST-PENN STA,2184.0,76.317696
619,Monday,JKSN HT-ROOSVLT,2111.0,71.301136
383,Monday,145 ST,1841.0,54.228496
427,Monday,34 ST-HERALD SQ,1776.0,50.466816
579,Monday,FLUSHING-MAIN,1552.0,38.539264
595,Monday,GRD CNTRL-42 ST,1453.0,33.779344
475,Monday,86 ST,1389.0,30.869136
545,Monday,CHURCH AV,1355.0,29.376400


In [146]:
# [[40.749719,-73.987823], [40.700377,-73.950234],[40.751776,-73.976848], [40.622687,-74.028398], [40.644041,-73.979678]]

In [147]:
# [[40.754222,-73.984569]] * 15

In [148]:
import math

In [149]:
monday_scales = day_of_week_final['dScale'][day_of_week_final['DAYOFWEEK'] == 'Monday'].head(10)
monday_scales = [int(i) for i in monday_scales]

print monday_scales

[248, 85, 76, 71, 54, 50, 38, 33, 30, 29]


In [150]:
monday_coords =[[40.754222, -73.984569], [40.8111094, -73.952343], [40.750373,-73.991057], [40.7495674, -73.98795], [40.7466444, -73.891338], [40.8247834, -73.944216],[40.749719,-73.987823], [40.700377,-73.950234],[40.751776,-73.976848], [40.622687,-74.028398], [40.650843,-73.949575]]

# monday_scales = [15,9,8,6,5]
# monday_coords_scaled = []
# for i in range(len(monday_scales):
#     monday_coords_scaled.append(list(itertools.repeat(monday_coord[i], monday_scales[i])))
    
# print monday_coords_scaled + [1,2]

In [151]:
print [monday_coords[0]] * 3 + [monday_coords[1]] * 3

[[40.754222, -73.984569], [40.754222, -73.984569], [40.754222, -73.984569], [40.8111094, -73.952343], [40.8111094, -73.952343], [40.8111094, -73.952343]]


In [152]:
print [] + monday_coords[0]

[40.754222, -73.984569]


In [153]:
def ScaleCoordinates(coords, scales):
    new_array = []
    for i in range(len(scales)):
        new_array = new_array + ([coords[i]] * scales[i])
    return new_array

In [158]:
def Jitter(arr, jitter=0.0005):
    jitter_list = []
    for row in arr:
        jitter_list = jitter_list + ([[row[0] + np.random.normal(-jitter, +jitter), row[1] +np.random.uniform(-jitter, +jitter)]])
    return jitter_list

In [159]:
monday_scaled_coordinates = ScaleCoordinates(monday_coords,monday_scales)
print scaled_coordinates[:5]

[[40.754222, -73.984569], [40.754222, -73.984569], [40.754222, -73.984569], [40.754222, -73.984569], [40.754222, -73.984569]]


In [160]:
monday_jitter_coords = Jitter(scaled_coordinates)

In [161]:
gmaps.heatmap(jitter_coords)

In [ ]:
# monday_coords = []
# monday_coords.append([[40.754222,-73.984569]] * 15)
# monday_coords.append([[40.8111094,-73.952343]] * 9)
# monday_coords.append([[40.7495674,-73.98795]] * 8)
# monday_coords.append([[40.7466444,-73.891338]] * 5)
# monday_coords.append([[40.8247834,-73.944216]] * 3)

# #                 [[40.754222,-73.984569]] * 15,
# #                 [[40.8111094,-73.952343]] * 9,
# #                 [[40.7495674,-73.98795]] * 8,
# #                 [[40.7466444,-73.891338]] * 5,
# #                 [[40.8247834,-73.944216]] * 3
# #                 ]
# # monday_scales = [15,9,8,6,5]
# # monday_coords_scaled = []
# # for i in monday_scales:
# #     for n in range(i):
# #         monday_coord_scaled.append(monday,)
# #                 [15 * [40.754222,-73.984569],
# #                 9 * [40.8111094,-73.952343],
# #                 9 * [40.7495674,-73.98795],
# #                 8 * [40.7466444,-73.891338],
# #                 8 * [40.8247834,-73.944216]]


# print monday_coords

# print ('/n' + 'break')

# monday_coord_jitter = Jitter(monday_coord)
# print monday_coord_jitter

In [ ]:
# gmaps.heatmap(monday_coord_jitter)

In [ ]:
# df_locations = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv')
# df_locations = df_locations[['Station Name', 'Station Latitude', 'Station Longitude']]
# df_locations_upper = df_locations
# df_locations_upper['Station Name'] = df_locations['Station Name'].str.upper()
# day_of_week_final.merge(df_locations_upper, left_on='STATION', right_on='Station Name', how='outer')